In [13]:
from openai import OpenAI
client = OpenAI(api_key='input your api key')

In [14]:
file = client.files.create(
file=open("Intake_Form.pdf", "rb"),
purpose='assistants'
)

In [15]:
assistant = client.beta.assistants.create(
instructions = "Role and Context: You are an AI assistant for an attorney specializing in real estate property tax. You have access to a standard intake form called Intake_Form.pdf. Task Overview: When a user interacts with you, 1. identify and list all the required fields (blank spaces) in the Intake_Form.pdf; 2.Ask the user to provide the information needed for those fields; 3.If the user cannot provide all the required details, fill in only the fields for which information was provided; 4.Leave blank any fields for which no information was provided, ensuring the overall layout and formatting remain consistent with the original form; 5.Generate a PDF (partial or complete) that includes: All supplied information in the correct fields and Any remaining blank fields clearly left empty (with the same label/formatting); 6.Generate user_data.json to store the provided information with the name of the field as keys and the provided information as values. Important Requirements: 1.Always preserve the original form's format, labels, and structure; 2.If certain fields are not filled out, those fields must remain visibly blank in the final document; 3.You must never remove or alter any existing text, labels, or structure in the form; 4.The final PDF output should contain all provided information in the correct fields, even if some fields remain empty; 5. Remember to generate user_data.json to store already provided information.",
model="gpt-4o",
tools=[{"type": "code_interpreter"}],
tool_resources={
  "code_interpreter": {
    "file_ids": [file.id]
  }
}
)

In [16]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_QeEqgAIqLnqhuSXIk90bKBwP', created_at=1738003704, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [17]:
message = client.beta.threads.messages.create(
thread_id=thread.id,
role="user",
content="I need an attorney in property tax. Can you help me?"
)

In [19]:
run = client.beta.threads.runs.create_and_poll(
thread_id=thread.id,
assistant_id=assistant.id
)

In [20]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
for message in reversed(messages.data):
    print(message.role + ': ' + message.content[0].text.value)


user: I need an attorney in property tax. Can you help me?
assistant: I specialize in assisting with real estate property tax, particularly with tasks related to completing and handling legal forms associated with property tax issues. Let's start by reviewing the uploaded document to see what details are required. Then, I can help you fill out the form with the information you provide. Let's identify the required fields in the Intake_Form.pdf.
assistant: The text extracted from the PDF provides an overview of the fields in the form. Based on the text preview, here are some of the fields likely present in the form:

1. County
2. Township
3. PIN(s) (Property Index Number)
4. Address of Property
5. Square Feet of Building
6. Recent Purchase? (Yes/No)
7. Demolition? (Yes/No)
8. Demo Permit? (Yes/No)
9. Paid Receipt? (Yes/No)
10. Demo Affidavit? (Yes/No)
11. Excess Land? (Yes/No)
12. Non-Buildable Area (Conservancy, Wetlands, Detention Pond) (Yes/No)
13. New Building Completed When?
14. Hav

In [21]:
message = client.beta.threads.messages.create(
thread_id=thread.id,
role="user",
content="1. Cook; 2. Northfield. The rest is still unknown. Could you generate the form filled with these two blanks fielded?"
)

In [22]:
run = client.beta.threads.runs.create_and_poll(
thread_id=thread.id,
assistant_id=assistant.id
)

In [23]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
for message in reversed(messages.data):
    print(message.role + ': ' + message.content[0].text.value)




user: I need an attorney in property tax. Can you help me?
assistant: I specialize in assisting with real estate property tax, particularly with tasks related to completing and handling legal forms associated with property tax issues. Let's start by reviewing the uploaded document to see what details are required. Then, I can help you fill out the form with the information you provide. Let's identify the required fields in the Intake_Form.pdf.
assistant: The text extracted from the PDF provides an overview of the fields in the form. Based on the text preview, here are some of the fields likely present in the form:

1. County
2. Township
3. PIN(s) (Property Index Number)
4. Address of Property
5. Square Feet of Building
6. Recent Purchase? (Yes/No)
7. Demolition? (Yes/No)
8. Demo Permit? (Yes/No)
9. Paid Receipt? (Yes/No)
10. Demo Affidavit? (Yes/No)
11. Excess Land? (Yes/No)
12. Non-Buildable Area (Conservancy, Wetlands, Detention Pond) (Yes/No)
13. New Building Completed When?
14. Hav

In [24]:
print(messages.data[0])

Message(id='msg_0h0oDcxQZMW9NM5cNJB6TP4F', assistant_id='asst_LXFyxb5AyIWVk4aOgmyJadO6', attachments=[Attachment(file_id='file-RHbcewMc5NZxiUWvcEajaX', tools=[CodeInterpreterTool(type='code_interpreter')]), Attachment(file_id='file-CfBbPmFLraWoEVSXga2sbn', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=263, file_path=FilePath(file_id='file-CfBbPmFLraWoEVSXga2sbn'), start_index=223, text='sandbox:/mnt/data/filled_Intake_Form.pdf', type='file_path'), FilePathAnnotation(end_index=325, file_path=FilePath(file_id='file-RHbcewMc5NZxiUWvcEajaX'), start_index=293, text='sandbox:/mnt/data/user_data.json', type='file_path')], value='I have filled in the "County" and "Township" fields as requested and generated the PDF document. You can download the filled PDF and the JSON containing the provided data from the links below:\n\n- [Download Filled PDF Form](sandbox:/mnt/data/filled_Intake_Form.p

In [26]:
doc_names = ['user_data.json', 'filled_intake_form.pdf']
for idx in range(len(messages.data[0].attachments)):
        
    file_data = client.files.content(messages.data[0].attachments[idx].file_id)
    file_data_bytes = file_data.read()

    with open(doc_names[idx], "wb") as f:
            f.write(file_data_bytes)

In [12]:
file_data = client.files.content(messages.data[0].attachments[0].file_id)
file_data_bytes = file_data.read()

with open('filled_intake_form.pdf', "wb") as f:
            f.write(file_data_bytes)

In [37]:
thread = client.beta.threads.create()

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)
print(f"New run created: {run.id}")

while True:
    user_input = input("User: ")

    # If user says 'complete conversation', break out of the loop.
    if user_input.strip().lower() == "complete conversation":
        print("Ending conversation...")
        break

    # Create a new user message in the thread.
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input
    )

    # Poll for the assistant response within the same run.
    # This blocks until the assistant response is ready.
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    # Retrieve all messages in this thread to print them.
    messages = client.beta.threads.messages.list(thread_id=thread.id).data

    # Print the entire conversation (user & assistant).
    # For a real UI, you might only want to print the last message or so.
    print("\n--- Conversation so far ---")
    for msg in reversed(messages):
        for part in msg.content:
        # If there's a file in this part, you can download it
            if hasattr(part, 'file') and part.file is not None:
                file_id = part.file.id  # The unique ID for the file in the system
                file_name = part.file.filename  # The original filename, e.g. "Filled_Intake_Form.pdf"

                print(f"Downloading file ID {file_id} with name {file_name}...")

                # 3. Download the file data
                file_data = client.files.download(file_id=file_id)

                # 4. Write the data to a local file
                with open(file_name, "wb") as f:
                    f.write(file_data)

                print(f"Saved as {file_name}")    
        role = msg.role
        # Some messages can have multiple "parts," but typically you only have text in one block:
        content = msg.content[0].text.value if msg.content and msg.content[0].text else ''
        print(f"{role.capitalize()}: {content}")
    print("--------------------------\n")


New run created: run_RB2wU4iOpTYdaU9gMCSvTZKq

--- Conversation so far ---
Assistant: The Intake Form PDF has 8 blanks that need to be filled. Here is the context of the fields from the document:

1. County
2. Township
3. PIN(s)
4. Address of Property
5. Square Feet of Building
6. Recent Purchase?
7. Demolition?
8. Demolition Permit?

To fill out the form, please provide the required information for each of these fields.
User: Hi
Assistant: Hello! How can I assist you today? Are you ready to provide the information needed to fill out the Intake Form PDF?
--------------------------


--- Conversation so far ---
Assistant: The Intake Form PDF has 8 blanks that need to be filled. Here is the context of the fields from the document:

1. County
2. Township
3. PIN(s)
4. Address of Property
5. Square Feet of Building
6. Recent Purchase?
7. Demolition?
8. Demolition Permit?

To fill out the form, please provide the required information for each of these fields.
User: Hi
Assistant: Hello! How c